![](http://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/UPM/EscPolitecnica/EscUpmPolit_p.gif "UPM")

# Trabajo final SITC
## Análisis de sentimientos en Twitter

Departamento de Ingeniería de Sistemas Telemáticos. Universidad Politécnica de Madrid.

Realizado por:
- Juan Bermudo Mera
- Margarita Bolívar Jiménez
- Lourdes Fernández Nieto
- Ramón Pérez Hernández

© 2017

# Clustering aplicado sobre el fichero de tweets

## Tabla de contenidos

* [Preparación del dataset](#1.-Preparación-del-dataset)
	* [Apertura del fichero Excel y conversión a dataframe](#Apertura-del-fichero-Excel-y-conversión-a-dataframe)
    * [Tokenización y stemming](#Tokenización-y-stemming)
* [Clustering con K-means](#2.-Clustering-con-K-means)
    * [K-means con unigramas](#K-means-con-unigramas)
    * [K-means con unigramas y bigramas](#K-means-con-unigramas-y-bigramas)
    * [K-means con unigramas, bigramas y trigramas](#K-means-con-unigramas,-bigramas-y-trigramas)
    * [Asignación de temas/topics](#Asignación-de-temas/topics)
* [Topic modeling con LDA](#3.-Topic-modeling-con-LDA)

In [1]:
# Se importan todas las librerías necesarias (Algunas necesitan instalación con: pip install <nombre_paquete>)

# Procesado del fichero csv
import pandas as pd

# Tokenización y stemming
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Clustering
from sklearn.cluster import KMeans

# LDA
from gensim import corpora, models

## 1. Preparación del dataset

* ### Apertura del fichero Excel y conversión a dataframe

In [2]:
tweets = pd.read_excel('ficheros/Preprocesados/tweets_spainGeo.xlsx',header=0,encoding='iso8859_15')

In [3]:
# Se comprueba que se ha cargado correctamente
tweets.head()

,content,Latitude,Longitude
0,"Regalo para socios: 'Talento a la fuga', el li...",40.416,-3.703
1,"La recuperación del ‘ladrillo’ dispara el 144,...",40.416,-3.703
2,TELEVISIÓN SERIES - Cuenta atrás para conocer ...,40.416,-3.703
3,Nuestra taza de hoy es un homenaje a uno de lo...,40.416,-3.703
4,@MCadepe nos vemos el 19 de mayooo!!!,40.416,-3.703


In [4]:
tweets.shape

(26363, 3)

* ### Tokenización y stemming

In [5]:
# Se descargan las palabras de parada (stopwords) en español 
nltk.download("stopwords")
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/monphdez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Se obtienen los signos de puntuación de la librería punctuation
non_words = list(punctuation)

# Añadimos signos de puntuación que se usan en español
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))

In [7]:
# Se definen las funciones para realizar la tokenización y el stemming
stemmer = SnowballStemmer('spanish')
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # Eliminamos lo que no sean palabras
    text = ''.join([c for c in text if c not in non_words])
    # Tokenización
    tokens = tknzr.tokenize(text)

    # Stemming
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

## 2. Clustering con K-means

In [8]:
# Definición de la función que realiza el clustering con K-means y muestra los resultados
def kmeans_function(final_cluster, mx_tfidf, tweets, vectorizer, text, r_state):
    for num_clusters in range(5,final_cluster):
        km = KMeans(n_clusters=num_clusters, random_state=r_state)
        km.fit(mx_tfidf)  
        clusters = km.labels_.tolist()  
        tweets["Cluster_" + text + "_" + str(num_clusters)] = clusters
        print('Número de clusters: %d' % (num_clusters))
        print(tweets["Cluster_" + text + "_" + str(num_clusters)].value_counts())

        # Impresión de palabras más frecuentes por grupo.
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]
        for i in range(num_clusters):
            print("Cluster {} : Palabras más frecuentes :".format(i))
            for ind in order_centroids[i, :10]: 
                print(' %s' % vectorizer.get_feature_names()[ind])
            print('')
        print('')

In [9]:
# Extraemos los tweets y los guardamos en una lista
tweets_text = tweets["content"].tolist()

* ### K-means con unigramas

In [10]:
# Convertimos la lista en la matriz Tf-idf
vectorizer_unig = TfidfVectorizer(
                max_df = 0.95,
                max_features = 30000,
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                use_idf = True,
                stop_words = spanish_stopwords,
                ngram_range=(1,1))
mx_tfidf_unig = vectorizer_unig.fit_transform(tweets_text)

In [11]:
# Palabras más frecuentes con las condiciones marcadas (pequeño subconjunto; sólo se verá 
# aquí para mostrar que se han extraido correctamente las features)
print(vectorizer_unig.get_feature_names()[100:110])

['abrel', 'abren', 'abrepuert', 'abres', 'abri', 'abriend', 'abriendomadr', 'abriendopuert', 'abrig', 'abril']


In [12]:
mx_tfidf_unig.shape

(26363, 30000)

In [13]:
# Realización del clustering con k-means
kmeans_function(16, mx_tfidf_unig, tweets, vectorizer_unig, "unig", 3333)

Número de clusters: 5
0    17365
4     6714
3      974
2      711
1      599
Name: Cluster_unig_5, dtype: int64
Cluster 0 : Palabras más frecuentes :
 madr
 par
 in
 mas
 spain
 at
 com
 tod
 si
 im

Cluster 1 : Palabras más frecuentes :
 dgtmadr
 dgt
 nivel
 autov
 pk
 autop
 retencion
 amarill
 decrecient
 crecient

Cluster 2 : Palabras más frecuentes :
 tendenci
 ahor
 spain
 madr
 tiempodejueg
 visavis
 nadal
 sv
 manoslimpi
 fatim

Cluster 3 : Palabras más frecuentes :
 buen
 dias
 seguidor
 unfollowers
 tuitutil
 nuev
 retir
 viern
 madr
 jardin

Cluster 4 : Palabras más frecuentes :
 …
 par
 mas
 tod
 hoy
 madr
 com
 si
 dia
 hac


Número de clusters: 6
1    15172
2     6753
0     2512
4      712
5      615
3      599
Name: Cluster_unig_6, dtype: int64
Cluster 0 : Palabras más frecuentes :
 par
 …
 madr
 tod
 mejor
 mas
 dia
 com
 hac
 hoy

Cluster 1 : Palabras más frecuentes :
 madr
 mas
 spain
 si
 com
 via
 tod
 nuev
 mejor
 graci

Cluster 2 : Palabras más frecuentes :
 …
 bu

In [14]:
# Nueva prueba de clustering con k-means, al detectarse otros grupos con otra semilla
kmeans_function(16, mx_tfidf_unig, tweets, vectorizer_unig, "unig2", 4444)

Número de clusters: 5
0    22282
3     2467
4      712
1      599
2      303
Name: Cluster_unig2_5, dtype: int64
Cluster 0 : Palabras más frecuentes :
 …
 madr
 mas
 buen
 tod
 com
 in
 si
 hoy
 dia

Cluster 1 : Palabras más frecuentes :
 dgtmadr
 dgt
 nivel
 autov
 pk
 autop
 retencion
 amarill
 decrecient
 crecient

Cluster 2 : Palabras más frecuentes :
 list
 reserv
 whatsapp
 vip
 sab
 tartuf
 mejor
 show
 sal
 info

Cluster 3 : Palabras más frecuentes :
 par
 …
 madr
 tod
 mas
 dia
 com
 hac
 buen
 hoy

Cluster 4 : Palabras más frecuentes :
 tendenci
 ahor
 spain
 madr
 tiempodejueg
 visavis
 nadal
 sv
 manoslimpi
 fatim


Número de clusters: 6
4    15143
3     6245
0     1837
2     1551
5      988
1      599
Name: Cluster_unig2_6, dtype: int64
Cluster 0 : Palabras más frecuentes :
 madr
 tendenci
 ahor
 spain
 in
 at
 im
 comun
 españ
 aeropuert

Cluster 1 : Palabras más frecuentes :
 dgtmadr
 dgt
 nivel
 autov
 pk
 autop
 retencion
 amarill
 decrecient
 crecient

Cluster 2 : Pal

In [15]:
# Se comprueba que las columnas se han añadido correctamente
tweets.head()

,content,Latitude,Longitude,Cluster_unig_5,Cluster_unig_6,Cluster_unig_7,Cluster_unig_8,Cluster_unig_9,Cluster_unig_10,Cluster_unig_11,...,Cluster_unig2_6,Cluster_unig2_7,Cluster_unig2_8,Cluster_unig2_9,Cluster_unig2_10,Cluster_unig2_11,Cluster_unig2_12,Cluster_unig2_13,Cluster_unig2_14,Cluster_unig2_15
0,"Regalo para socios: 'Talento a la fuga', el li...",40.416,-3.703,0,0,0,0,1,5,3,...,4,0,3,8,0,10,10,1,13,7
1,"La recuperación del ‘ladrillo’ dispara el 144,...",40.416,-3.703,0,1,0,6,1,5,9,...,4,0,3,8,0,0,0,0,0,3
2,TELEVISIÓN SERIES - Cuenta atrás para conocer ...,40.416,-3.703,0,0,0,0,1,5,3,...,4,0,3,8,0,10,10,1,13,7
3,Nuestra taza de hoy es un homenaje a uno de lo...,40.416,-3.703,0,1,0,6,2,4,9,...,4,1,3,4,9,0,11,0,11,0
4,@MCadepe nos vemos el 19 de mayooo!!!,40.416,-3.703,0,1,0,6,1,5,9,...,4,0,3,8,0,0,0,0,0,3


* ### K-means con unigramas y bigramas

In [16]:
# Convertimos la lista en la matriz Tf-idf
vectorizer_unibi = TfidfVectorizer(
                max_df = 0.95,
                min_df = 0.00005,
                max_features = 30000,
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                use_idf = True,
                stop_words = spanish_stopwords,
                ngram_range=(1,2))
mx_tfidf_unibi = vectorizer_unibi.fit_transform(tweets_text)

In [17]:
mx_tfidf_unibi.shape

(26363, 30000)

In [18]:
# Realización del clustering con k-means
kmeans_function(16, mx_tfidf_unibi, tweets, vectorizer_unibi, "unibi", 3333)

Número de clusters: 5
1    18115
0     6850
4      703
3      590
2      105
Name: Cluster_unibi_5, dtype: int64
Cluster 0 : Palabras más frecuentes :
 …
 par
 mas
 tod
 madr
 hoy
 com
 dia
 si
 buen

Cluster 1 : Palabras más frecuentes :
 madr
 par
 buen
 in
 mas
 com
 spain
 tod
 si
 in madr

Cluster 2 : Palabras más frecuentes :
 unfollowers tuitutil
 nuev seguidor
 seguidor unfollowers
 unfollowers
 seguidor
 tuitutil
 nuev
 ey
 facult cc
 factor

Cluster 3 : Palabras más frecuentes :
 autop autov
 autov
 autop
 dgtmadr dgt
 dgtmadr
 dgt
 retencion nivel
 retencion
 nivel
 pk

Cluster 4 : Palabras más frecuentes :
 ahor tendenci
 tendenci
 ahor
 tendenci spain
 tendenci madr
 spain
 madr
 visavis ahor
 tiempodejueg
 tiempodejueg ahor


Número de clusters: 6
1    19776
4     3563
2      984
5      747
0      703
3      590
Name: Cluster_unibi_6, dtype: int64
Cluster 0 : Palabras más frecuentes :
 ahor tendenci
 tendenci
 ahor
 tendenci spain
 tendenci madr
 spain
 madr
 visavis ahor

* ### K-means con unigramas, bigramas y trigramas

In [19]:
# Convertimos la lista en la matriz Tf-idf
vectorizer_unibitri = TfidfVectorizer(
                max_df = 0.95,
                min_df = 0.00005,
                max_features = 30000,
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                use_idf = True,
                stop_words = spanish_stopwords,
                ngram_range=(1,3))
mx_tfidf_unibitri = vectorizer_unibitri.fit_transform(tweets_text)

In [20]:
mx_tfidf_unibitri.shape

(26363, 30000)

In [21]:
# Realización del clustering con k-means
kmeans_function(16, mx_tfidf_unibitri, tweets, vectorizer_unibitri, "unibitri", 3333)

Número de clusters: 5
1    17647
0     6642
3      791
2      696
4      587
Name: Cluster_unibitri_5, dtype: int64
Cluster 0 : Palabras más frecuentes :
 …
 par
 madr
 mas
 tod
 hoy
 com
 dia
 si
 nuestr

Cluster 1 : Palabras más frecuentes :
 madr
 par
 buen
 mas
 com
 si
 tod
 spain
 dias
 via

Cluster 2 : Palabras más frecuentes :
 ahor tendenci
 tendenci
 ahor
 tendenci spain
 ahor tendenci spain
 tendenci madr
 ahor tendenci madr
 spain
 madr
 visavis ahor

Cluster 3 : Palabras más frecuentes :
 in
 in madr
 madr
 at
 im at
 im
 comun madr
 comun
 madr comun
 madr comun madr

Cluster 4 : Palabras más frecuentes :
 autop autov
 autov
 autop
 dgtmadr dgt
 dgtmadr
 dgt
 retencion nivel
 retencion
 nivel
 pk


Número de clusters: 6
0    20706
1     2400
4     1470
5      696
3      587
2      504
Name: Cluster_unibitri_6, dtype: int64
Cluster 0 : Palabras más frecuentes :
 …
 madr
 mas
 com
 si
 in
 hoy
 graci
 spain
 dia

Cluster 1 : Palabras más frecuentes :
 par
 …
 tuitutil
 nuev

* ### Asignación de temas/topics

In [22]:
# Por defecto, asignamos el tema 'otros'
tweets['Topic'] = 'otros'

In [23]:
# Asignación de tema 'sitios-madrid'
tweets.loc[(tweets["Cluster_unig_8"] == 4) & (tweets["Cluster_unig_11"] == 7) & 
           (tweets["Cluster_unig_12"] == 7) & (tweets["Cluster_unig_13"] == 1) & 
           (tweets["Cluster_unig_15"] == 12) & (tweets["Cluster_unig2_10"] == 5) &
           (tweets["Cluster_unig2_11"] == 8) & (tweets["Cluster_unig2_12"] == 8) &
           (tweets["Cluster_unig2_13"] == 5) & (tweets["Cluster_unig2_14"] == 8) &
           (tweets["Cluster_unig2_15"] == 4) & (tweets["Cluster_unibi_11"] == 8) & 
           (tweets["Cluster_unibi_14"] == 1) & (tweets["Cluster_unibitri_12"] == 9) &
           (tweets["Cluster_unibitri_13"] == 6),'Topic'] = 'sitios-madrid'

In [24]:
tweets['Topic'].value_counts()

otros            26307
sitios-madrid       56
Name: Topic, dtype: int64

In [25]:
# Se incluye un nuevo subconjunto a 'sitios-madrid', que teóricamente incluye pueblos de la Comunidad
tweets.loc[(tweets["Cluster_unig_14"] == 4) | (tweets["Cluster_unig_15"] == 0) | 
           (tweets["Cluster_unibi_10"] == 2) | (tweets["Cluster_unibitri_12"] == 10),
           'Topic'] = 'sitios-madrid'

In [26]:
tweets['Topic'].value_counts()

otros            25269
sitios-madrid     1094
Name: Topic, dtype: int64

In [27]:
tweets.loc[tweets["Topic"] == 'sitios-madrid',["content"]].head()

,content
2781,I'm at @Facilisimo.com in Villaviciosa de Odón...
3828,I'm at AC Hotel La Finca in Pozuelo de Alarcón...
4824,¡La cocina es el corazón de la casa! Refórmala...
5845,"I'm at Madrid in Madrid, España"
6208,I'm at Viva Gym embajadores


In [28]:
# Asignación de tema 'buenos-dias'
tweets.loc[(tweets["Cluster_unig_7"] == 1) & (tweets["Cluster_unig_11"] == 5) & 
           (tweets["Cluster_unig_13"] == 8) & (tweets["Cluster_unig_14"] == 13) & 
           (tweets["Cluster_unig_15"] == 10) & (tweets["Cluster_unig2_6"] == 5) &
           (tweets["Cluster_unig2_8"] == 1) & (tweets["Cluster_unig2_9"] == 5) &
           (tweets["Cluster_unig2_10"] == 8) & (tweets["Cluster_unig2_11"] == 10) &
           (tweets["Cluster_unig2_12"] == 10) & (tweets["Cluster_unig2_14"] == 10) &
           (tweets["Cluster_unig2_15"] == 8) & (tweets["Cluster_unibi_7"] == 2) & 
           (tweets["Cluster_unibi_9"] == 7) & (tweets["Cluster_unibi_12"] == 11) & 
           (tweets["Cluster_unibi_13"] == 2) & (tweets["Cluster_unibi_14"] == 5) & 
           (tweets["Cluster_unibi_15"] == 14) & (tweets["Cluster_unibitri_6"] == 2) &
           (tweets["Cluster_unibitri_7"] == 3) & (tweets["Cluster_unibitri_9"] == 8) &
           (tweets["Cluster_unibitri_12"] == 8) & (tweets["Cluster_unibitri_13"] == 1),
           'Topic'] = 'buenos-dias'

In [29]:
tweets['Topic'].value_counts()

otros            24865
sitios-madrid     1091
buenos-dias        407
Name: Topic, dtype: int64

In [30]:
tweets.loc[tweets["Topic"] == 'buenos-dias',["content"]].head()

,content
413,"@kardiez2000 Buenos días, no sé si conocías la..."
687,@margarita_5239 Buenos días cielete y feliz fi...
699,@maricarmengar15 @Pedramuz @Mariquinaperez @ma...
845,@raulitopro11 buenos dias Raul feliz Viernes ...
1066,"@Vermutinn jajajajjaja Se lo merecía, es insu..."


In [31]:
# Asignación de tema 'retiro'
tweets.loc[(tweets["Cluster_unibi_14"] == 12) & (tweets["Cluster_unibitri_9"] == 4) & 
           (tweets["Cluster_unibitri_10"] == 7) & (tweets["Cluster_unibitri_11"] == 7) & 
           (tweets["Cluster_unibitri_14"] == 7) & (tweets["Cluster_unibitri_15"] == 7), 
           'Topic'] = 'retiro'

In [32]:
tweets['Topic'].value_counts()

otros            24805
sitios-madrid     1090
buenos-dias        407
retiro              61
Name: Topic, dtype: int64

In [33]:
# Es muy restrictivo, cuando había un grupo claro en Cluster_unibi_14. Usaremos solo ese
tweets.loc[(tweets["Cluster_unibi_14"] == 12), 'Topic'] = 'retiro'

In [34]:
tweets['Topic'].value_counts()

otros            24743
sitios-madrid     1088
buenos-dias        407
retiro             125
Name: Topic, dtype: int64

In [35]:
tweets.loc[tweets["Topic"] == 'retiro',["content"]].head()

,content
9832,"""Claro que lo bueno cuesta que no lo ganas si ..."
10039,ayer eramos rookies hoy somos all-stars @ Parq...
10330,De Jardines del Buen Retiro \n#mrcade #reproch...
10438,"#Beerlover #Instabeer - #Madrid, #Spain. 🇪🇸 @ ..."
10500,#TBT Mayo 15. Palacio de Cristal en el Parque ...


In [36]:
# Asignación de tema 'gran-via'
tweets.loc[(tweets["Cluster_unibi_13"] == 4) & (tweets["Cluster_unibi_14"] == 6) &
           (tweets["Cluster_unibi_15"] == 0) & (tweets["Cluster_unibitri_12"] == 3), 'Topic'] = 'gran-via'

In [37]:
tweets['Topic'].value_counts()

otros            24633
sitios-madrid     1083
buenos-dias        407
retiro             125
gran-via           115
Name: Topic, dtype: int64

In [38]:
tweets.loc[tweets["Topic"] == 'gran-via',["content"]].head()

,content
72,Shredded Chicken Salpicón via @Leslie_Limon
728,vía @giphy
1719,Alimentaria celebra en el recinto Gran Vía de ...
3717,Ayer me compré de oferta estos 3 libros en las...
5531,Vía @LaCastaSusana


In [39]:
# Asignación de tema 'gracias'
tweets.loc[(tweets["Cluster_unig_14"] == 11) & (tweets["Cluster_unig2_9"] == 6) &
           (tweets["Cluster_unig2_14"] == 12) & (tweets["Cluster_unibi_13"] == 11) & 
           (tweets["Cluster_unibi_15"] == 11), 'Topic'] = 'gracias'

In [40]:
tweets['Topic'].value_counts()

otros            24577
sitios-madrid     1083
buenos-dias        407
retiro             125
gran-via           115
gracias             56
Name: Topic, dtype: int64

In [41]:
tweets.loc[tweets["Topic"] == 'gracias',["content"]].head()

,content
233,@GlamourSpain correspondiente porque no lo he ...
270,Y se cumple un año de #SeisHermanas muchas gra...
449,"Mi principe, muchas gracias, siempre apoyando,..."
643,@Tri_finisher @totibest Muchas gracias por com...
1287,@stephipugliese @BrunoVD @FerRubioA @soyAlicia...


In [42]:
# Asignación de tema 'followers'
tweets.loc[(tweets["Cluster_unibi_5"] == 2) & (tweets["Cluster_unibi_8"] == 0) & 
           (tweets["Cluster_unibi_10"] == 0) & (tweets["Cluster_unibi_11"] == 10) & 
           (tweets["Cluster_unibi_12"] == 0) & (tweets["Cluster_unibi_13"] == 7) & 
           (tweets["Cluster_unibi_15"] == 1) & (tweets["Cluster_unibitri_6"] == 1) & 
           (tweets["Cluster_unibitri_8"] == 3) & (tweets["Cluster_unibitri_9"] == 0) & 
           (tweets["Cluster_unibitri_10"] == 0) & (tweets["Cluster_unibitri_11"] == 0) & 
           (tweets["Cluster_unibitri_12"] == 11) & (tweets["Cluster_unibitri_9"] == 0) & 
           (tweets["Cluster_unibitri_13"] == 3) & (tweets["Cluster_unibitri_14"] == 12) & 
           (tweets["Cluster_unibitri_15"] == 12), 'Topic'] = 'followers'

In [43]:
tweets['Topic'].value_counts()

otros            24472
sitios-madrid     1083
buenos-dias        407
retiro             125
gran-via           115
followers          105
gracias             56
Name: Topic, dtype: int64

In [44]:
tweets.loc[tweets["Topic"] == 'followers',["content"]].head()

,content
418,"Nuevos seguidores: 2, unfollowers: 1 (14:45) #..."
562,"Nuevos seguidores: 1, unfollowers: 1 (13:37) #..."
1003,"Nuevos seguidores: 1, unfollowers: 0 (11:20) #..."
1116,"Nuevos seguidores: 0, unfollowers: 1 (11:19) #..."
1170,"Nuevos seguidores: 1, unfollowers: 0 (11:19) #..."


In [45]:
# Asignación de tema 'tendencias-twitter'
tweets.loc[(tweets["Cluster_unig_5"] == 2) & (tweets["Cluster_unig_6"] == 4) & 
           (tweets["Cluster_unig_7"] == 5) & (tweets["Cluster_unig_8"] == 7) & 
           (tweets["Cluster_unig_9"] == 8) & (tweets["Cluster_unig_10"] == 9) &
           (tweets["Cluster_unig_11"] == 8) & (tweets["Cluster_unig_12"] == 10) &
           (tweets["Cluster_unig_13"] == 9) & (tweets["Cluster_unig_14"] == 12) &
           (tweets["Cluster_unig_15"] == 4) &  (tweets["Cluster_unig2_5"] == 4) &
           (tweets["Cluster_unig2_7"] == 5) & (tweets["Cluster_unig2_8"] == 5) &
           (tweets["Cluster_unig2_9"] == 1) & (tweets["Cluster_unig2_10"] == 7) &
           (tweets["Cluster_unig2_11"] == 4) & (tweets["Cluster_unig2_12"] == 4) &
           (tweets["Cluster_unig2_13"] == 7) & (tweets["Cluster_unig2_14"] == 4) &
           (tweets["Cluster_unig2_15"] == 6),
           'Topic'] = 'tendencias-twitter'

In [46]:
tweets['Topic'].value_counts()

otros                 23762
sitios-madrid          1083
tendencias-twitter      710
buenos-dias             407
retiro                  125
gran-via                115
followers               105
gracias                  56
Name: Topic, dtype: int64

In [47]:
tweets.loc[tweets["Topic"] == 'tendencias-twitter',["content"]].head()

,content
6,'nicholson' es ahora una tendencia en #Madrid
6992,8 Tendencias de Email marketing |
7734,Seguro de Vida; ahora!
8896,8 Tendencias de Email marketing |
9693,#cuentamenoche es ahora una tendencia en #Madrid


In [48]:
# Asignación de tema 'discoteca'
tweets.loc[(tweets["Cluster_unig_9"] == 7) & (tweets["Cluster_unig_11"] == 0) &
           (tweets["Cluster_unig2_6"] == 2) & (tweets["Cluster_unig2_8"] == 0) &
           (tweets["Cluster_unig2_10"] == 3) & (tweets["Cluster_unig2_12"] == 11) &
           (tweets["Cluster_unig2_13"] == 4) & (tweets["Cluster_unig2_14"] == 11) &
           (tweets["Cluster_unig2_15"] == 13),'Topic'] = 'discoteca'

In [49]:
tweets['Topic'].value_counts()

otros                 23591
sitios-madrid          1083
tendencias-twitter      710
buenos-dias             407
discoteca               171
retiro                  125
gran-via                115
followers               105
gracias                  56
Name: Topic, dtype: int64

In [50]:
tweets.loc[tweets["Topic"] == 'discoteca',["content"]].head()

,content
327,Listas #show meclub! Te esperamos con la mejor...
328,Zoe- RESERVADOS y Listas! - SÁBADO- 100 eu re...
332,No te pierdas este finde en en Riviera Madrid ...
347,Este sabado la mejor fiesta con mejor espectac...
360,"Este sabado SHOW ME' en La Tartufo!! TRAP, #FU..."


In [51]:
# Asignación de tema 'trafico' (como aparece en todos, cogeremos el primero)
tweets.loc[(tweets["Cluster_unig_5"] == 1),'Topic'] = 'trafico'

In [52]:
tweets['Topic'].value_counts()

otros                 22992
sitios-madrid          1083
tendencias-twitter      710
trafico                 599
buenos-dias             407
discoteca               171
retiro                  125
gran-via                115
followers               105
gracias                  56
Name: Topic, dtype: int64

In [53]:
tweets.loc[tweets["Topic"] == 'trafico',["content"]].head()

,content
2549,#RETENCIÓN nivel AMARILLO en AUTOPISTA / AUTOV...
2568,#RETENCIÓN nivel AMARILLO en AUTOPISTA / AUTOV...
2869,#RETENCIÓN nivel AMARILLO en AUTOPISTA / AUTOV...
2873,#RETENCIÓN nivel ROJO en AUTOPISTA / AUTOVÍA #...
3148,#RETENCIÓN nivel AMARILLO en AUTOPISTA / AUTOV...


In [54]:
# Asignación de tema 'aeropuerto'
tweets.loc[(tweets["Cluster_unig_14"] == 1) & (tweets["Cluster_unig_15"] == 5) &
           (tweets["Cluster_unibi_7"] == 3) & (tweets["Cluster_unibi_14"] == 13),
           'Topic'] = 'aeropuerto'

In [55]:
tweets['Topic'].value_counts()

otros                 22880
sitios-madrid          1083
tendencias-twitter      710
trafico                 599
buenos-dias             407
discoteca               171
retiro                  125
gran-via                115
aeropuerto              112
followers               105
gracias                  56
Name: Topic, dtype: int64

In [56]:
tweets.loc[tweets["Topic"] == 'aeropuerto',["content"]].head()

,content
8034,I'm at Aeropuerto Adolfo Suárez Madrid-Barajas...
9904,Pues ya estamos aquí (@ Aeropuerto Adolfo Suár...
10525,I'm at Aeropuerto Adolfo Suárez Madrid-Barajas...
10621,I'm at Aeropuerto Adolfo Suárez Madrid-Barajas...
10973,Back (@ Aeropuerto Adolfo Suárez Madrid-Baraja...


In [57]:
# Repetiremos el ejercicio de clustering con el subconjunto catalogado como 'otros', para ver que
# no hay más clústeres triviales
ot_tweets = tweets.loc[tweets["Topic"] == 'otros',["content","Latitude","Longitude"]]
ot_tweets.shape

(22880, 3)

In [58]:
ot_tweets.head()

,content,Latitude,Longitude
0,"Regalo para socios: 'Talento a la fuga', el li...",40.416,-3.703
1,"La recuperación del ‘ladrillo’ dispara el 144,...",40.416,-3.703
2,TELEVISIÓN SERIES - Cuenta atrás para conocer ...,40.416,-3.703
3,Nuestra taza de hoy es un homenaje a uno de lo...,40.416,-3.703
4,@MCadepe nos vemos el 19 de mayooo!!!,40.416,-3.703


In [59]:
# Extraemos los tweets y los guardamos en una lista
ot_tweets_text = ot_tweets["content"].tolist()

In [60]:
# Convertimos la lista en la matriz Tf-idf
vectorizer_unig_ot = TfidfVectorizer(
                max_df = 0.95,
                max_features = 30000,
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                use_idf = True,
                stop_words = spanish_stopwords,
                ngram_range=(1,1))
mx_tfidf_unig_ot = vectorizer_unig_ot.fit_transform(ot_tweets_text)

In [61]:
mx_tfidf_unig_ot.shape

(22880, 29442)

In [62]:
# Realización del clustering con k-means. No se detectan nuevas agrupaciones relevantes
kmeans_function(11, mx_tfidf_unig_ot, ot_tweets, vectorizer_unig_ot, "uni_ot", 3333)

Número de clusters: 5
0    12166
1     4933
3     2255
2     2206
4     1320
Name: Cluster_uni_ot_5, dtype: int64
Cluster 0 : Palabras más frecuentes :
 mas
 com
 si
 via
 per
 graci
 dia
 buen
 quier
 hac

Cluster 1 : Palabras más frecuentes :
 …
 mas
 hoy
 com
 si
 dia
 hac
 mejor
 nuestr
 vid

Cluster 2 : Palabras más frecuentes :
 par
 …
 mas
 mejor
 dia
 hoy
 com
 hac
 nuestr
 madr

Cluster 3 : Palabras más frecuentes :
 madr
 in
 …
 españ
 real
 palaci
 plaz
 mas
 adecc
 dia

Cluster 4 : Palabras más frecuentes :
 tod
 …
 par
 sab
 graci
 dias
 buen
 mejor
 per
 madr


Número de clusters: 6
5    11906
1     4479
2     2296
4     1850
3     1245
0     1104
Name: Cluster_uni_ot_6, dtype: int64
Cluster 0 : Palabras más frecuentes :
 si
 …
 quier
 com
 per
 par
 ver
 pued
 mas
 hac

Cluster 1 : Palabras más frecuentes :
 …
 par
 mas
 com
 nuev
 nuestr
 mejor
 vid
 stigmab
 buen

Cluster 2 : Palabras más frecuentes :
 madr
 in
 …
 españ
 real
 palaci
 plaz
 mas
 adecc
 par

Cluster 3 

In [63]:
# Se exportan los tweets con la nueva columna a un fichero Excel
tweets[['content','Latitude','Longitude','Topic']].to_excel('ficheros/TweetsConTopic/tweets_spainGeo_topic.xlsx', header=True, index=False)

## 3. Topic modeling con LDA

In [64]:
# Se redefine la función de tokenización para tener en cuenta las palabras de parada, sin hacer stemming
# (para tener más términos para buscar los temas)
def tokenize_stop(text):
    # Eliminamos lo que no sean palabras
    text = ''.join([c for c in text if c not in non_words])
    # Eliminamos las palabras de parada
    text = ''.join([c for c in text if c not in spanish_stopwords])

    # Tokenización
    tokens = tknzr.tokenize(text)
    
    return tokens

In [65]:
# Generamos una lista de tweets donde cada tweet sea una lista de palabras
words_tweet = [tokenize_stop(tweet) for tweet in tweets_text]
# Se genera el diccionario
dictionary = corpora.Dictionary(words_tweet)
# Se convierte el diccionario en BOW (Bag of Words)
tweet_term_matrix = [dictionary.doc2bow(doc) for doc in words_tweet]
# Lanzamos el modelo (tarda bastante, más de 1 hora)
ldamodel = models.ldamodel.LdaModel(tweet_term_matrix, num_topics=8, id2word = dictionary, passes=100)

# Impresión de resultados (no hay mucha precisión en los resultados; sólo está claro el tema de 'trafico',
# que es el Topic 5, y hay ciertos indicios del tema 'tendencias' y 'aeropuerto' en el Topic 7, pero no
# se distinguen).
for topic in ldamodel.show_topics(num_topics=10, formatted=False, num_words=10):
    print("Topic {}: Words: ".format(topic[0]))
    topicwords = [w for (w, val) in topic[1]]
    print(topicwords)

Topic 0: Words: 
['d', 'l', 'dl', 'Mdrid', '…', 'ls', 'El', 'n', 'cn', 'un']
Topic 1: Words: 
['d', 'l', 'n', '…', 'pr', 'qu', 'ls', 'un', 'nivl', 's']
Topic 2: Words: 
['Mdrid', 'in', 'd', 'n', 'l', 'D', 'ls', 'cn', '…', 'Sl']
Topic 3: Words: 
['l', 'qu', 's', 'n', '…', 'd', 'pr', 'un', 'ls', 'm']
Topic 4: Words: 
['…', 'd', 'l', 'n', 'pr', 'cn', 'ES', 'Stigmbs', 'Mdrid', 'L']
Topic 5: Words: 
['n', 'l', 'DGT', 'pk', 'DGTMdrid', 'RETENCIÓN', 'AUTOPISTA', 'AUTOVÍA', 'AMARILLO', 'A']
Topic 6: Words: 
['…', 'd', 'dís', 'Buns', 'mdrid', 'pr', 'fin', 'smn', 'ls', 'virns']
Topic 7: Words: 
['Mdrid', 'n', 'un', 's', 'hr', 'tndnci', 't', 'Im', 'd', 'Arpurt']


In [10]:
# Se repetirá el proceso haciendo también stemming.
words_tweet_stem = [tokenize(tweet) for tweet in tweets_text]
dictionary_stem = corpora.Dictionary(words_tweet_stem)
tweet_term_matrix_stem = [dictionary_stem.doc2bow(doc) for doc in words_tweet_stem]
ldamodel_stem = models.ldamodel.LdaModel(tweet_term_matrix_stem, num_topics=8, id2word = dictionary_stem, passes=100)

# Impresión de resultados (no hay mucha precisión en los resultados nuevamente; el único claro vuelve a ser
# 'trafico', que es el Topic 7).
for topic_stem in ldamodel_stem.show_topics(num_topics=10, formatted=False, num_words=10):
    print("Topic {}: Words: ".format(topic_stem[0]))
    topicwords_stem = [w for (w, val) in topic_stem[1]]
    print(topicwords_stem)

Topic 0: Words: 
['de', 'buen', 'a', 'dias', 'palaci', 'el', 'un', 'los', 'graci', 'adolf']
Topic 1: Words: 
['que', 'no', '…', 'lo', 'y', 'a', 'te', 'es', 'spain', 'si']
Topic 2: Words: 
['…', 'y', 'de', 'con', 'dia', 'del', 'el', 'madr', 'par', 'un']
Topic 3: Words: 
['de', 'la', 'el', 'en', '…', 'a', 'y', 'los', 'que', 'las']
Topic 4: Words: 
['de', '…', 'la', 'y', 'que', 'a', 'el', 'me', 'en', 'mi']
Topic 5: Words: 
['de', 'madr', 'la', 'el', 'en', 'in', '…', 'del', 'españ', 'las']
Topic 6: Words: 
['en', 'es', 'una', 'ahor', 'madr', 'tendenci', 'nuev', 'estadi', 'aeropuert', 'suarez']
Topic 7: Words: 
['en', 'al', 'nivel', 'pk', 'dgt', 'retencion', 'dgtmadr', 'autop', 'autov', 'amarill']


<hr>

## Licencia

El notebook está licenciado libremente bajo la licencia [Creative Commons Attribution Share-Alike](https://creativecommons.org/licenses/by/2.0/).

La base del código empleado procede del trabajo de Yogesh Kulkarni llamado [Sentiment Analysis of Twitter Posts on Chennai Floods using Python](https://www.analyticsvidhya.com/blog/2017/01/sentiment-analysis-of-twitter-posts-on-chennai-floods-using-python/) y del trabajo de Brandon Rose llamado [Document Clustering with Python](http://brandonrose.org/clustering)

© 2017 - Juan Bermudo Mera, Margarita Bolívar Jiménez, Lourdes Fernández Nieto, Ramón Pérez Hernández.

Universidad Politécnica de Madrid.